<h1><center>ОТБОР ПРИЗНАКОВ</center></h1>

Вернёмся к модели «Космолайна», предсказывающей отток клиентов. Вы уже обучили её базовую версию, а также вариант модели с дополнительными признаками. Чтобы улучшить качество предсказаний модели и увеличить скорость её работы в продакшене, избавьтесь от лишних признаков, используя методы, изученные в предыдущем уроке.

В начале определите глобальные переменные. Вот шаблон того, как это можно сделать:

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg
import mlflow
from mlflow.tracking import MlflowClient
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, StandardScaler,
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

import catboost as cb
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder

TABLE_NAME = "clean_users_churn" # таблица с данными
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "sign_selection" # название эксперимента
RUN_NAME = "sign_selection" 
REGISTRY_MODEL_NAME = "sign_selection" # название зарегистрированной модели 
FS_ASSETS = "fs_assets" 

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
    else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

Далее загрузите данные для задачи оттока из таблицы в базе данных PostgreSQL и положите их в переменную df.

In [ ]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

После того как вы выделили категориальные колонки, закодируйте их для подачи в вашу модель:

**Обучим модель**. Подготовим данные:


In [ ]:
# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]


Обучим модель:

In [ ]:

# Разделим данные на две части - для обучения и для проверки качества предсказания:
X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target']) 

# Тренировочная выборка
cat_features_tr = X_tr.select_dtypes(include='object')
potential_binary_features_tr = cat_features_tr.nunique() == 2

binary_cat_features_tr = cat_features_tr[potential_binary_features_tr[potential_binary_features_tr].index]
other_cat_features_tr = cat_features_tr[potential_binary_features_tr[~potential_binary_features_tr].index]
num_features_tr = X_tr.select_dtypes(['float'])

# Валидационная выборка
cat_features_val = X_val.select_dtypes(include='object')
potential_binary_features_val = cat_features_val.nunique() == 2

binary_cat_features_val = cat_features_val[potential_binary_features_val[potential_binary_features_val].index]
other_cat_features_val = cat_features_val[potential_binary_features_val[~potential_binary_features_val].index]
num_features_val = X_val.select_dtypes(['float'])

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = other_cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Определим список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer( [ ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
                                    ('cat', CatBoostEncoder(), non_binary_cat_cols),
                                    ('num', StandardScaler(), num_cols) ],verbose_feature_names_out=False )

# Трансформируем исходные данные с помощью созданного preprocessor
X_tr_transformed = preprocessor.fit_transform(X_tr, y_tr)
X_val_transformed = preprocessor.transform(X_val)

# Создадим модель:
model = CatBoostClassifier()

# Обучим модель:
model.fit(X_tr_transformed, y_tr)

Проверим качество нашей модели:

In [ ]:
# Сделаем предсказание:
y_val_pred = model.predict(X_val_transformed)

# Определим точность:
accuracy = str(accuracy_score(y_val, y_val_pred))[:4]
print(f"Точность модели: {accuracy}")

Напомним, что отбор признаков важно проводить на обучающем наборе данных, чтобы исключить переобучение.
<br>Для отбора признаков используйте библиотеку mlxtend и методы Sequential Forward Selection (SFS) и Sequential Backward Selection (SBS).

---

**Задание 1**

Отберите десять лучших признаков при помощи методов SFS и SBS. Используйте такие настройки:
- параметр кросс-валидации равен 4;
- функция floating=False, то есть выключена;
- в качестве оценщика используйте алгоритм случайного леса c числом деревьев 300, а основной метрикой будет roc_auc.

Имена отобранных признаков сохраните в переменные top_sfs и top_sbs.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

estimator = RandomForestClassifier(n_estimators=300) # ваш код здесь #
print('estimator')

sfs = SFS(estimator, k_features=10, forward=True, floating=False, scoring='roc_auc', cv=4, n_jobs=-1) # ваш код здесь #
print('sfs')
sbs = SFS(estimator, k_features=10, forward=False, floating=False, scoring='roc_auc', cv=4, n_jobs=-1) # ваш код здесь #
print('sbs')

sfs = sfs.fit(X_tr_transformed, y_tr) # ваш код здесь # запустите отбор признаков
print('sfs2')
sbs = sbs.fit(X_tr_transformed, y_tr) # ваш код здесь #
print('sbs2')

top_sfs = sfs.k_feature_names_ # ваш код здесь #
print('top_sfs')
top_sbs = sbs.k_feature_names_ # ваш код здесь #
print('top_sbs')

print('\nSequential Forward Selection (k=10)')
print('CV Score:')
print(sfs.k_score_)

print('\nSequential Backward Selection')
print('CV Score:')
print(sbs.k_score_)

Отбор признаков — тоже исследование, и его результаты важно сохранять. После того как вы запустили sfs.fit() или sbs.fit(), на выходе получаются объекты sfs_df и sbs_df, из которых просто получить таблицу с результатами отбора. Сделать это можно так: 

In [ ]:
sfs_df = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
sbs_df = pd.DataFrame.from_dict(sbs.get_metric_dict()).T 

Сохраните этот датафрейм в CSV-файл для каждого из методов отбора признаков:

In [ ]:
# os.mkdir(FS_ASSETS)

sfs_df.to_csv(f"{FS_ASSETS}/sfs.csv")
sbs_df.to_csv(f"{FS_ASSETS}/sbs.csv") 

Постройте графики отбора признаков для методов SFS и SBS и сохраните графики в директорию с артефактами.
Начнём с SFS. 

In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig = plot_sfs(sfs.get_metric_dict(), kind='std_dev')

plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()

plt.savefig(f"{FS_ASSETS}/sfs.png") 

График визуализирует процесс последовательного прямого отбора признаков SFS с указанием стандартного отклонения StdDev. По оси x — количество признаков, по оси y — производительность модели. Очевидно, что с увеличением числа признаков производительность повышается, особенно заметен рост при добавлении первых нескольких признаков. Но примерно после добавления пяти признаков улучшение замедляется и стабилизируется.


Теперь график SBS.

In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig = plot_sfs(sbs.get_metric_dict(), kind='std_dev')

plt.title('Sequential Backward Selection (w. StdDev)')
plt.grid()
plt.show()

plt.savefig(f"{FS_ASSETS}/sbs.png") 

**Задание 2**

Создайте два списка list в Python, в которых будут храниться признаки, отобранные путём пересечения и объединения. Файл для пересекающихся множеств назовите interc_features, для объединяющихся — union_features.
Для проверки решения вернитесь в это задание и нажмите кнопку Проверить.

In [ ]:
interc_features = list(set(top_sbs) & set(top_sfs))
union_features = list(set(top_sbs) | set(top_sfs))

In [ ]:
# experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

# with mlflow.start_run(run_name=f"{RUN_NAME}", experiment_id=experiment_id) as run:
#     run_id = run.info.run_id
   
#     mlflow.log_artifacts(FS_ASSETS)

Регистрируем:

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

TRACKING_SERVER_HOST = '127.0.0.1'
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(model, REGISTRY_MODEL_NAME)
    model_registred_name = REGISTRY_MODEL_NAME
    mlflow.register_model("runs:/{}/model".format(run.info.run_id), model_registred_name)
    # model_version_id = mlflow.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id
    mlflow.log_artifacts(FS_ASSETS)

**Задание 4**


Теперь обучите две версии модели на двух наборах признаков и сравните метрики. Зарегистрируйте обе модели в MLflow в экспериментах feature_selection_union и feature_selection_intersection. В переменные ниже вставьте соответствующую информацию.

Укажем название эксперимента и название **первой модели и версии запуска**:

In [ ]:

EXPERIMENT_NAME = "feature_selection" # название эксперимента

REGISTRY_MODEL_NAME = "feature_selector" # название зарегистрированной модели 
RUN_NAME = "feature_selection_union" 

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
    else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

Обучим первую версию модели:

In [ ]:
# Разделим данные на две части - для обучения и для проверки качества предсказания:
X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target']) 

# Тренировочная выборка
cat_features_tr = X_tr.select_dtypes(include='object')
potential_binary_features_tr = cat_features_tr.nunique() == 2

binary_cat_features_tr = cat_features_tr[potential_binary_features_tr[potential_binary_features_tr].index]
other_cat_features_tr = cat_features_tr[potential_binary_features_tr[~potential_binary_features_tr].index]
num_features_tr = X_tr.select_dtypes(['float'])

# Валидационная выборка
cat_features_val = X_val.select_dtypes(include='object')
potential_binary_features_val = cat_features_val.nunique() == 2

binary_cat_features_val = cat_features_val[potential_binary_features_val[potential_binary_features_val].index]
other_cat_features_val = cat_features_val[potential_binary_features_val[~potential_binary_features_val].index]
num_features_val = X_val.select_dtypes(['float'])

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = other_cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Определим список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer( [ ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
                                    ('cat', CatBoostEncoder(), non_binary_cat_cols),
                                    ('num', StandardScaler(), num_cols) ],verbose_feature_names_out=False )

# Трансформируем исходные данные с помощью созданного preprocessor
X_tr_transformed = preprocessor.fit_transform(X_tr, y_tr)
X_val_transformed = preprocessor.transform(X_val)

# Создадим модель:
model = CatBoostClassifier()

# Обучим модель:
model.fit(X_tr_transformed, y_tr)

Регистрируем:

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

TRACKING_SERVER_HOST = '127.0.0.1'
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(model, REGISTRY_MODEL_NAME)
    model_registred_name = REGISTRY_MODEL_NAME
    mlflow.register_model("runs:/{}/model".format(run.info.run_id), model_registred_name)
    # model_version_id = mlflow.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id
    mlflow.log_artifacts(FS_ASSETS)

Укажем название эксперимента и **вторую версию запуска**:

In [ ]:

EXPERIMENT_NAME = "feature_selection" # название эксперимента

REGISTRY_MODEL_NAME = "feature_selector" # название зарегистрированной модели 
RUN_NAME = "feature_selection_intersection" 

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
    else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

Обучим первую версию модели:

In [ ]:
# Разделим данные на две части - для обучения и для проверки качества предсказания:
X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target']) 

# Тренировочная выборка
cat_features_tr = X_tr.select_dtypes(include='object')
potential_binary_features_tr = cat_features_tr.nunique() == 2

binary_cat_features_tr = cat_features_tr[potential_binary_features_tr[potential_binary_features_tr].index]
other_cat_features_tr = cat_features_tr[potential_binary_features_tr[~potential_binary_features_tr].index]
num_features_tr = X_tr.select_dtypes(['float'])

# Валидационная выборка
cat_features_val = X_val.select_dtypes(include='object')
potential_binary_features_val = cat_features_val.nunique() == 2

binary_cat_features_val = cat_features_val[potential_binary_features_val[potential_binary_features_val].index]
other_cat_features_val = cat_features_val[potential_binary_features_val[~potential_binary_features_val].index]
num_features_val = X_val.select_dtypes(['float'])

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = other_cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Определим список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer( [ ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
                                    ('cat', CatBoostEncoder(), non_binary_cat_cols),
                                    ('num', StandardScaler(), num_cols) ],verbose_feature_names_out=False )

# Трансформируем исходные данные с помощью созданного preprocessor
X_tr_transformed = preprocessor.fit_transform(X_tr, y_tr)
X_val_transformed = preprocessor.transform(X_val)

# Создадим модель:
model = CatBoostClassifier()

# Обучим модель:
model.fit(X_tr_transformed, y_tr)

Регистрируем:

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

TRACKING_SERVER_HOST = '127.0.0.1'
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(model, REGISTRY_MODEL_NAME)
    model_registred_name = REGISTRY_MODEL_NAME
    mlflow.register_model("runs:/{}/model".format(run.info.run_id), model_registred_name)
    # model_version_id = mlflow.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id
    mlflow.log_artifacts(FS_ASSETS)

In [ ]:
'''В базе данных playground_mle_20250529_05fed48463 ожидается две записи по указанным параметрам, найдено - 0. Проверяющий запрос:

select * 
from runs join model_versions on runs.run_uuid=model_versions.run_id 
where runs.name in ('feature_selector', 'feature_selector') and 
      runs.run_uuid in ('feature_selection_intersection', 'feature_selection_union') 
      and model_versions.name = 'feature_selector' and version in (1, 1)'''